In [1]:
import os


import tensorflow as tf
from tensorflow import keras

import numpy as np

2024-01-02 18:10:54.990003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 18:10:54.990032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 18:10:54.990945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 18:10:54.996615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 18:10:55.764782: W tensorflow/compiler/tf2

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   243k      0  0:05:37  0:05:37 --:--:--  260k9M    0     0   208k      0  0:06:34  0:00:58  0:05:36  222k


In [3]:
!tar -xf aclImdb_v1.tar.gz

In [4]:
!ls aclImdb/

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls aclImdb/test/

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train/

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [8]:
!cat aclImdb/train/pos/10000_8.txt

Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days without th

In [9]:
!rm -rf aclImdb/train/unsup/

In [12]:
batch_size = 32

raw_train_ds = keras.utils.text_dataset_from_directory('./aclImdb/train/',
                                                      validation_split=.2,
                                                      batch_size=batch_size,
                                                      subset='training',
                                                      seed=0)

raw_val_ds = keras.utils.text_dataset_from_directory('./aclImdb/train/',
                                                    validation_split=.2,
                                                     batch_size=batch_size,
                                                     subset='validation',
                                                     seed=0)

raw_test_ds = keras.utils.text_dataset_from_directory('./aclImdb/test/',
                                                     batch_size=batch_size)

print(f'Number of batches in raw_train_ds: {raw_train_ds.cardinality()}')
print(f'Number of batches in raw_val_ds: {raw_val_ds.cardinality()}')
print(f'Number of batches in raw_test_ds: {raw_test_ds.cardinality()}')

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [16]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b"This was really a very bad movie. I am a huge fan of Italian Horror, Argento, Mario Bava, Fulci and yes, even our good friend here Lamberto sometimes comes out with a good one. I found the first two 'Demons' films to be highly entertaining - they were so bad they were great but this one is just so bad that it is really, really bad. It is intensely boring, the story never goes anywhere and I hated the characters - the wife slapping husband and whiny cry-baby pain in the *** wife drove me mad, there was nowhere near enough of the story devoted to the Ogre who was probably the best actor in the whole film. I turned it off about three quarters of the way through because I was very, very BORED! Don't bother."
0
b"This movie is an utter waste of time, the plot is awful, the dialogue is awful.<br /><br />The acting is OK, but the actors have absolutely no plot or script to work with. The photography and some of the special effects are OK, too, but again there is nothing interesting in this 

In [19]:
import string
import re

def custom_standization(input_data):
    lowercase = tf.strings.lower(input_data)
    striped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        striped_html, f"[{re.escape(string.punctuation)}]", ""
    )


max_features = 20000
embbeding_dim = 128
sequence_lenght = 500

vectorize_layer = keras.layers.TextVectorization(standardize=custom_standization,
                                                 max_tokens=max_features,
                                                 output_mode='int',
                                                 output_sequence_length=sequence_lenght
                                                )

text_ds = raw_train_ds.map(lambda x, y: x)

vectorize_layer.adapt(text_ds)

In [22]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)


In [24]:
inputs = keras.Input(shape=(None,), dtype='int64')

x = keras.layers.Embedding(max_features, embbeding_dim)(inputs)

x = keras.layers.Dropout(.5)(x)

x = keras.layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)

x = keras.layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)

x = keras.layers.GlobalMaxPooling1D()(x)

x = keras.layers.Dense(128, activation='relu')(x)

x = keras.layers.Dropout(.5)(x)

predictions = keras.layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = keras.Model(inputs, predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')


In [25]:
history = model.fit(train_ds, epochs=3, validation_data=val_ds)

Epoch 1/3


2024-01-02 19:07:23.963746: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-01-02 19:07:24.150325: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-02 19:07:24.717342: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f170c5bd300 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-02 19:07:24.717363: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-01-02 19:07:24.727710: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1704233244.810346    4627 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 35s 50ms/step - loss: 0.5105 - accuracy: 0.7054 - val_loss: 0.3127 - val_accuracy: 0.8588
Epoch 2/3
625/625 [==============================] - 5s 9ms/step - loss: 0.2310 - accuracy: 0.9093 - val_loss: 0.3213 - val_accuracy: 0.8746
Epoch 3/3
625/625 [==============================] - 5s 9ms/step - loss: 0.1166 - accuracy: 0.9590 - val_loss: 0.4695 - val_accuracy: 0.8680


In [26]:
model.evaluate(test_ds)

782/782 [==============================] - 2s 3ms/step - loss: 0.4853 - accuracy: 0.8599


[0.48526668548583984, 0.8599200248718262]